# Importando as Bibliotecas

In [1566]:
# import pandas
import pandas as pd
import seaborn as sn

In [1567]:
import spacy

In [1568]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# from imblearn.over_sampling import SMOTE

In [1569]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [1570]:
import re

# Conhecendo os dados

In [1571]:
df = pd.read_csv(filepath_or_buffer="data/Dataset_5971.csv", sep=",")

In [1572]:
display(df)

,LABEL,TEXT,URL,EMAIL,PHONE
0,ham,Your opinion about me? 1. Over 2. Jada 3. Kusr...,No,No,No
1,ham,What's up? Do you want me to come online? If y...,No,No,No
2,ham,So u workin overtime nigpun?,No,No,No
3,ham,"Also sir, i sent you an email about how to log...",No,No,No
4,Smishing,Please Stay At Home. To encourage the notion o...,No,No,No
...,...,...,...,...,...
5966,ham,Becoz its &lt;#&gt; jan whn al the post ofic...,No,No,No
5967,ham,Its a valentine game. . . send dis msg to all ...,No,No,No
5968,ham,We r outside already.,No,No,No
5969,ham,The Xmas story is peace.. The Xmas msg is love...,No,No,No


In [1573]:
df.dtypes

LABEL    object
TEXT     object
URL      object
EMAIL    object
PHONE    object
dtype: object

In [1574]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5971 entries, 0 to 5970
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   LABEL   5971 non-null   object
 1   TEXT    5971 non-null   object
 2   URL     5971 non-null   object
 3   EMAIL   5971 non-null   object
 4   PHONE   5971 non-null   object
dtypes: object(5)
memory usage: 233.4+ KB


In [1575]:
df.describe()

,LABEL,TEXT,URL,EMAIL,PHONE
count,5971,5971,5971,5971,5971
unique,5,5950,2,2,2
top,ham,FREE MESSAGE Activate your 500 FREE Text Messa...,No,No,No
freq,4843,2,5764,5952,5126


In [1576]:
df["LABEL"].unique()

array(['ham', 'Smishing', 'spam', 'Spam', 'smishing'], dtype=object)

In [1577]:
df["URL"].unique()

array(['No', 'yes'], dtype=object)

In [1578]:
df["EMAIL"].unique()

array(['No', 'yes'], dtype=object)

In [1579]:
df["PHONE"].unique()

array(['No', 'yes'], dtype=object)

In [1580]:
len([x for x in df.duplicated() if x == True])

17

In [1581]:
i = 0
index_dup = []
for x in df.duplicated():
    if x == True:
        index_dup.append(i)
    i += 1

In [1582]:
df.iloc[index_dup]

,LABEL,TEXT,URL,EMAIL,PHONE
1351,Smishing,UR GOING 2 BAHAMAS! CallFREEFONE 08081560665 a...,No,No,yes
1803,Smishing,Someone U know has asked our dating service 2 ...,No,No,yes
2962,ham,He remains a bro amongst bros,No,No,No
3691,ham,Reverse is cheating. That is not mathematics.,No,No,No
3873,ham,The sign of maturity is not when we start sayi...,No,No,No
3907,Smishing,U have a secret admirer. REVEAL who thinks U R...,No,No,yes
3908,Smishing,Congrats! 2 mobile 3G Videophones R yours. cal...,No,No,yes
4141,ham,The greatest test of courage on earth is to be...,No,No,No
4730,ham,"Sorry,in meeting I'll call later",No,No,No
4926,Smishing,URGENT!! Your 4* Costa Del Sol Holiday or £500...,No,No,yes


In [1583]:
df[df["TEXT"] == "Sorry,in meeting I'll call later"]

,LABEL,TEXT,URL,EMAIL,PHONE
416,ham,"Sorry,in meeting I'll call later",No,No,No
4730,ham,"Sorry,in meeting I'll call later",No,No,No


In [1584]:
df[df["TEXT"] == "Arun can u transfr me d amt"]

,LABEL,TEXT,URL,EMAIL,PHONE
414,ham,Arun can u transfr me d amt,No,No,No
5342,ham,Arun can u transfr me d amt,No,No,No


# Transformando os dados

In [1585]:
df_transformado = df.copy()

In [1586]:
df_transformado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5971 entries, 0 to 5970
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   LABEL   5971 non-null   object
 1   TEXT    5971 non-null   object
 2   URL     5971 non-null   object
 3   EMAIL   5971 non-null   object
 4   PHONE   5971 non-null   object
dtypes: object(5)
memory usage: 233.4+ KB


In [1587]:
df_transformado["LABEL"].unique()

array(['ham', 'Smishing', 'spam', 'Spam', 'smishing'], dtype=object)

In [1588]:
df_transformado["LABEL"].replace("Spam", "spam", inplace=True)
df_transformado["LABEL"].replace("Smishing", "smishing", inplace=True)
df_transformado["LABEL"].replace(["ham", "spam", "smishing"], [-1, 0, 1], inplace=True)

In [1589]:
df_transformado["URL"].replace(["No", "yes"], [0, 1], inplace=True)
df_transformado["EMAIL"].replace(["No", "yes"], [0, 1], inplace=True)
df_transformado["PHONE"].replace(["No", "yes"], [0, 1], inplace=True)

In [1590]:
display(df_transformado["URL"].unique())
display(df_transformado["EMAIL"].unique())
display(df_transformado["PHONE"].unique())

array([0, 1])

array([0, 1])

array([0, 1])

In [1591]:
df_transformado.drop(["URL", "EMAIL", "PHONE"], axis=1, inplace=True)

In [1592]:
len([x for x in df_transformado.duplicated() if x == True]) ## Ele aumenta por causa do tratamento que se fez para as palvras minusculas

20

In [1593]:
df_transformado.drop_duplicates(inplace=True)

In [1594]:
display(len(df_transformado))
display(len(df))

display(len(df) - len(df_transformado))

5951

5971

20

In [1595]:
df_transformado.reset_index(drop=True, inplace=True)

In [1596]:
display(df.head())
display(df_transformado.head())

,LABEL,TEXT,URL,EMAIL,PHONE
0,ham,Your opinion about me? 1. Over 2. Jada 3. Kusr...,No,No,No
1,ham,What's up? Do you want me to come online? If y...,No,No,No
2,ham,So u workin overtime nigpun?,No,No,No
3,ham,"Also sir, i sent you an email about how to log...",No,No,No
4,Smishing,Please Stay At Home. To encourage the notion o...,No,No,No


,LABEL,TEXT
0,-1,Your opinion about me? 1. Over 2. Jada 3. Kusr...
1,-1,What's up? Do you want me to come online? If y...
2,-1,So u workin overtime nigpun?
3,-1,"Also sir, i sent you an email about how to log..."
4,1,Please Stay At Home. To encourage the notion o...


# Analisando os dados

In [1597]:
display(len(df_transformado[df_transformado["LABEL"] == -1]))
display(len(df_transformado[df_transformado["LABEL"] == 0]))
display(len(df_transformado[df_transformado["LABEL"] == 1]))

4833

489

629

# NLP

#### 1500 para treino e o resto para validação

In [1598]:
# !python -m spacy download en_core_web_trf

In [1599]:
nlp = spacy.load("en_core_web_trf")

In [1600]:
def remover_stopwords(text):
    doc = nlp(text)
    frases_sem_stopwords = " ".join([token.text for token in doc if not token.is_stop])
    return frases_sem_stopwords

In [1601]:
def remover_espacos(text):
    frases_sem_espaco_frente = " ".join(text.split())
    return frases_sem_espaco_frente

In [1602]:
def remover_caractere_especial(text):
    texto_sem_cs = re.sub(r"ï¿½", r"", text)
    texto_sem_cs = re.sub(r"&lt;([^>]+)&gt;", r"", texto_sem_cs)
    texto_sem_cs = re.sub(r"�", r" ", texto_sem_cs)
    texto_sem_cs = re.sub(r"ü", r"u", texto_sem_cs)
    texto_sem_cs = re.sub(r"Ü", r"U", texto_sem_cs)
    texto_sem_cs = re.sub(r"\(", r" ", texto_sem_cs)
    texto_sem_cs = re.sub(r"&amp;", r"", texto_sem_cs)
    return texto_sem_cs

In [1603]:
def remover_emoji(text):
    text_sem_emoji = re.sub(r"((:|;)(-){0,1}(\)|\())", f"", text)
    return text_sem_emoji

In [1604]:
def remover_pontos(text):
    doc = nlp(text)
    frases_sem_ponto = " ".join([token.text for token in doc if not token.is_punct])
    return frases_sem_ponto

In [1605]:
def lemmatizar(text):
    doc = nlp(text)
    frases_lematizadas = " ".join([token.lemma_ for token in doc])
    return frases_lematizadas

In [1606]:
def substituir_numero_telefone(text):
    texto_com_numero_padronizado = re.sub(r"((\d{4}\s\d{3}\s\d{4})|\d{10,12})(\s|$)", f"PH_NUMBER ", text)
    return texto_com_numero_padronizado

In [1607]:
def substituir_url(text):
    re_url = r"(http://|www.)\S+"
    texto_com_numero_padronizado = re.sub(re_url, f"URL", text)
    return texto_com_numero_padronizado

In [1608]:
def substituir_email(text):
    texto_com_email_padronizado = re.sub(r"\S+@\S+($){0,1}", f"EMAIL", text)
    return texto_com_email_padronizado

In [1609]:
def pre_processamento(text):
    texto_processado = text
    print(texto_processado)
    texto_processado = remover_emoji(texto_processado)
    texto_processado = remover_caractere_especial(texto_processado)
    texto_processado = substituir_numero_telefone(texto_processado)
    texto_processado = substituir_url(texto_processado)
    texto_processado = substituir_email(texto_processado)
    texto_processado = remover_espacos(texto_processado)  
    texto_processado = remover_pontos(texto_processado)
    texto_processado = lemmatizar(texto_processado)
    print(texto_processado)
    return texto_processado

In [1610]:
df_transformado["PROCESSED_TEXT"] = df_transformado["TEXT"].apply(pre_processamento)

Your opinion about me? 1. Over 2. Jada 3. Kusruthi 4. Lovable 5. Silent 6. Spl character 7. Not matured 8. Stylish 9. Simple Pls reply..
your opinion about I 1 over 2 Jada 3 Kusruthi 4 Lovable 5 Silent 6 spl character 7 not matured 8 stylish 9 simple pl reply
What's up? Do you want me to come online? If you are free we can talk sometime�
what be up do you want I to come online if you be free we can talk sometime
So u workin overtime nigpun?
so u workin overtime nigpun
Also sir, i sent you an email about how to log into the usc payment portal. I.ll send you another message that should explain how things are back home. Have a great weekend.
also sir I send you an email about how to log into the usc payment portal i.ll send you another message that should explain how thing be back home have a great weekend
Please Stay At Home. To encourage the notion of staying at home. All tax-paying citizens are entitled to �305.96 or more emergency refund. smsg.io/fCVbD
please stay at home to encourage

In [1637]:
i = 0
for x in df_transformado["LABEL"]:
    if x == 0:
        print(df_transformado["LABEL"].iloc[i])
        print(df_transformado["TEXT"].iloc[i])
        print(df_transformado["PROCESSED_TEXT"].iloc[i])
    i += 1

0
lyricalladie(21/F) is inviting you to be her friend. Reply YES-910 or NO-910. See her: www.SMS.ac/u/hmmross STOP? Send STOP FRND to 62468
lyricalladie 21 f be invite you to be her friend reply yes-910 or no-910 see her url stop send stop frnd to 62468
0
Hello from Orange. For 1 month's free access to games, news and sport, plus 10 free texts and 20 photo messages, reply YES. Terms apply: www.orange.co.uk/ow
hello from Orange for 1 month 's free access to game news and sport plus 10 free text and 20 photo message reply yes term apply url
0
New Offer! Save upto 40\% electricity bill with Power Saver(GOVT. LAB TESTED), Rs. 1050/-(free home delivery) 3 Yr. Guarantee Call 9891943823,9891943780
new offer save upto 40\% electricity bill with Power Saver govt lab test r 1050/- free home delivery 3 yr guarantee call 9891943823,ph_number
0
You will recieve your tone within the next 24hrs. For Terms and conditions please see Channel U Teletext Pg 750
you will recieve your tone within the next 2

In [1612]:
i = 0
for x in df_transformado["PROCESSED_TEXT"].duplicated():
    if x == True:
        print([df_transformado["LABEL"].iloc[i]])
        print(i, [df_transformado["PROCESSED_TEXT"].iloc[i]])
    i += 1

[1]
225 ['please call ph_number immediately as there be an urgent message wait for you']
[1]
320 ['do you want a new video handset 750 any time any network min unlimite text camcorder reply or call now ph_number for del Sat am']
[1]
431 ['please call ph_number immediately as there be an urgent message wait for you']
[1]
501 ['dear V0DAF0NE customer you be a 5,00,000 winner of guarantee Caller Prize this be our final attempt to contact you to claim call ph_number now']
[1]
549 ['please call ph_number immediately as there be an urgent message wait for you']
[1]
709 ['u have a secret admirer who be look 2 make contact with u find out who they r*reveal who think UR so special call on PH_NUMBER']
[1]
825 ['dear V0DAF0NE customer you be a 5,00,000 winner of guarantee Caller Prize this be our final attempt to contact you to claim call ph_number now']
[1]
869 ['PH_NUMBER URGENT your mobile no 07xxxxxxxxx win a £ 2,000 bonus caller prize on 02/06/03 this be the 2nd attempt to reach you call ph_

In [1613]:
i = 0
for x in df_transformado["PROCESSED_TEXT"] == "":
    if x == True:
        print([df_transformado["LABEL"].iloc[i]])
        print(i, [df_transformado["PROCESSED_TEXT"].iloc[i]])
    i += 1

[-1]
4472 ['']
[-1]
4772 ['']


In [1614]:
df_transformado.drop_duplicates(subset=["LABEL","PROCESSED_TEXT"], inplace=True)

In [1615]:
#Remover os que estão duplicados
#TODO: conferir se foram os melhores dados
# df_transformado.drop_duplicates(subset=["PROCESSED_TEXT"], inplace=True)

In [1616]:
df_transformado.head(20)

,LABEL,TEXT,PROCESSED_TEXT
0,-1,Your opinion about me? 1. Over 2. Jada 3. Kusr...,your opinion about I 1 over 2 Jada 3 Kusruthi ...
1,-1,What's up? Do you want me to come online? If y...,what be up do you want I to come online if you...
2,-1,So u workin overtime nigpun?,so u workin overtime nigpun
3,-1,"Also sir, i sent you an email about how to log...",also sir I send you an email about how to log ...
4,1,Please Stay At Home. To encourage the notion o...,please stay at home to encourage the notion of...
5,1,BankOfAmerica Alert 137943. Please follow http...,BankOfAmerica alert 137943 please follow url r...
6,-1,Sorry dude. Dont know how i forgot. Even after...,sorry dude do not know how I forget even after...
7,-1,I don't quite know what to do. I still can't g...,I do not quite know what to do I still can not...
8,-1,Ok lor. Anyway i thk we cant get tickets now c...,ok lor anyway I thk we can not get ticket now ...
9,-1,Wat r u doing now?,wat r u do now


In [1617]:
# TODO: Melhorar alguns lemmas, colocar stemm, retirar os caracteres especiais
display(df_transformado["PROCESSED_TEXT"].iloc[1])
display(df_transformado["TEXT"].iloc[1])

'what be up do you want I to come online if you be free we can talk sometime'

"What's up? Do you want me to come online? If you are free we can talk sometime�"

In [1618]:
display(len(df_transformado[df_transformado["LABEL"] == -1]))
display(len(df_transformado[df_transformado["LABEL"] == 0]))
display(len(df_transformado[df_transformado["LABEL"] == 1]))

4826

468

494

# Treinamento

In [1619]:
df_teste = df_transformado.iloc[:1500].copy()
df_validacao = df_transformado.iloc[1500:].copy()

In [1620]:
x = df_teste["PROCESSED_TEXT"]
y = df_teste["LABEL"]

In [1621]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)

In [1622]:
class_weights = {-1: 1, 0: 2, 1: 4}

In [1623]:
vectorizer = TfidfVectorizer()
x_train_tfidf = vectorizer.fit_transform(x_train)
x_test_tfidf = vectorizer.transform(x_test)

##### Logistic Regression

In [1624]:
model = LogisticRegression(solver="lbfgs", multi_class="multinomial", random_state=42, class_weight=class_weights)
model.fit(x_train_tfidf, y_train)

LogisticRegression(class_weight={-1: 1, 0: 2, 1: 4}, multi_class='multinomial',
                   random_state=42)

In [1625]:
y_pred = model.predict(x_test_tfidf)

In [1626]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")
print(f'Relatório de Classificação:\n{report}')

Acurácia do modelo: 0.94
Relatório de Classificação:
              precision    recall  f1-score   support

          -1       0.96      0.98      0.97       190
           0       1.00      0.44      0.61        16
           1       0.75      0.95      0.84        19

    accuracy                           0.94       225
   macro avg       0.90      0.79      0.81       225
weighted avg       0.95      0.94      0.94       225



#### Random Florest

In [1627]:
rf_classifier = RandomForestClassifier(random_state=42, class_weight=class_weights)
rf_classifier.fit(x_train_tfidf, y_train)

RandomForestClassifier(class_weight={-1: 1, 0: 2, 1: 4}, random_state=42)

In [1628]:
y_pred = rf_classifier.predict(x_test_tfidf)

In [1629]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")
print(f'Relatório de Classificação:\n{report}')

Acurácia do modelo: 0.93
Relatório de Classificação:
              precision    recall  f1-score   support

          -1       0.95      1.00      0.97       190
           0       0.80      0.25      0.38        16
           1       0.84      0.84      0.84        19

    accuracy                           0.93       225
   macro avg       0.86      0.70      0.73       225
weighted avg       0.93      0.93      0.92       225



#### SVM

In [1630]:
svm_classifier = SVC(kernel='linear',random_state=42, class_weight=class_weights)
svm_classifier.fit(x_train_tfidf, y_train)

SVC(class_weight={-1: 1, 0: 2, 1: 4}, kernel='linear', random_state=42)

In [1631]:
y_pred = svm_classifier.predict(x_test_tfidf)

In [1632]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")
print(f'Relatório de Classificação:\n{report}')

Acurácia do modelo: 0.94
Relatório de Classificação:
              precision    recall  f1-score   support

          -1       0.96      0.99      0.98       190
           0       0.89      0.50      0.64        16
           1       0.71      0.79      0.75        19

    accuracy                           0.94       225
   macro avg       0.86      0.76      0.79       225
weighted avg       0.94      0.94      0.93       225



# Validação

In [1633]:
x_v = vectorizer.transform(df_validacao["PROCESSED_TEXT"])
y_v = df_validacao["LABEL"]
y_pred = model.predict(x_v)

In [1634]:
x_v = vectorizer.transform(df_validacao["PROCESSED_TEXT"])
y_v = df_validacao["LABEL"]
y_pred = rf_classifier.predict(x_v)

In [1635]:
x_v = vectorizer.transform(df_validacao["PROCESSED_TEXT"])
y_v = df_validacao["LABEL"]
y_pred = svm_classifier.predict(x_v)

In [1636]:
accuracy = accuracy_score(y_v, y_pred)
report = classification_report(y_v, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")
print(f'Relatório de Classificação:\n{report}')

Acurácia do modelo: 0.95
Relatório de Classificação:
              precision    recall  f1-score   support

          -1       0.98      1.00      0.99      3606
           0       0.75      0.61      0.68       345
           1       0.78      0.77      0.77       337

    accuracy                           0.95      4288
   macro avg       0.84      0.79      0.81      4288
weighted avg       0.94      0.95      0.94      4288

